![logo](../files/misc/logo.png)
<h1 style="color:#872325">Decorators</h1>

## Modificando Funciones

### Fuciones como argumentos a funciones

Empezamos esta lectura recordando que en Python todo es un objeto: funciones, listas, numpy arrays, dicionarios, classes, etc. De igual manera, una función en Python puede obtener cualquier tipo de objeto. Por lo tanto, dentro de Python, **podemos definir funciones que tomen funciones**.

In [33]:
from typing import Callable
from math import log

def h(f: Callable[[float], float], x: float) -> float:
    return f(x ** 2)

h(log, 1.5)

0.8109302162163288

In [34]:
h

<function __main__.h(f:Callable[[float], float], x:float) -> float>

### Funciones dentro de funciones

Al igual que tomar funciones dentro de funciones, podemos escribir funciones dentro de funciones. Estas funciones son efímeras ya que dejan de existir una vez terminada la ejecución de la función

In [40]:
from random import randint, seed
seed(271828)

def f(x):
    def k(x, y):
        return x * y
    mult = randint(-100, 100)
    return k(x, mult)

for _ in range(3):
    print(f(2))

150
-122
-150


### Funciones que regresan funciones

¿De qué manera podríamos escribir una función que siempre multiplique `x` por un número aleatorio?

In [42]:
def f(x):
    def k(y):
        return x * y
    return k

In [46]:
h = f(3)
h(2)

6

In [65]:
def square_number(g):
    def k(x):
        return g(x ** 2)
    return k

def add_one(x):
    return x + 1

def inverse(x):
    return 1 / x

¿Cuál sería el resultado de ejecutar los siguiente programas?

In [ ]:
f = square_number(add_one)
f(3)

In [ ]:
g = square_number(inverse)
g(2)

In [ ]:
h = square_number(log)
h(2)

## Decoradores `@`

Para casos en los que deseemos modificar una función en base a alguna otra función, es común (y recomendable) usar **decoradores**. En otras palabras, 

> Un decorador en Python es una función que toma otra función y extiende su funcionalidad sin modificar la definición de la función original

<h2 style="color:teal">Ejemplo</h2>

Considerando la función `announce_func(f)`, escribamos un programa que regrese el valor de la función, imprimiendo en la pantalla que la función `f` se está corriendo actualmente.

In [81]:
def announce_func(f):
    print(f"Estas corriendo la función {f}")
    return f

In [82]:
# La manera "tradicional"
def f(x):
    return (x / 2) ** 2
annnounce_f = announce_func(f)
annnounce_f(10)

Estas corriendo la función <function f at 0x11522d8c8>


25.0

In [83]:
# Con un decorador
@announce_func
def announce_f(x):
    return (x / 2) ** 2

announce_f(10)

Estas corriendo la función <function announce_f at 0x11522dae8>


25.0

<h2 style="color:crimson">Ejercicios</h2>

**De regreso al ejemplo original**  
Recordemos la siguiente función:
```python
def square_number(g):
    def k(x):
        return g(x ** 2)
    return k

def add_one(x):
    return x + 1

def inverse(x):
    return 1 / x
```

1. Usando decoradores define la función `sq_add_one` que regrese la evaluación de `add_one` con `x`  evaluado en `square_number`.

---

2. Usando decoradores define la función `sq_inverse` que regrese la evaluación de `inverse` con `x`  evaluado en `square_number`.

## Acumulando Decoradores

Una ventaja de usar decoradores, desde un punto de vista estético, es la posibilidad de encadenar decoradores.

<h2 style="color:teal">Ejemplo</h2>

Consideramos la función `decorate` que, dado un elemento, lo decora de la siguiente manera:

In [104]:
def decorate(string):
    return f"<---{string}--->"

decorate(1)

'<---1--->'

Supongamos nos interesa crear una variante de `decorate` que considere una lista y nos regrese una nueva lista con `decorate` aplicado a cada uno de los elementos.

**Nota:** Este ejemplo es ilustrativo y puede ser solucionado muy fácilmente considerando una función que ocupe _list comprehensions_.

Evaluar decorate sobre una lista nos regresa lo siguiente:

In [106]:
decorate([1, 3])

'<---[1, 3]--->'

Introduciremos la función `vectorize` dentro de la librería `numpy` la cual toma una función `f` que regresa un único elemento y la transforma en una que aplica `f` a cada uno de los valores.

In [110]:
decorate_it = np.vectorize(f)
decorate_it([1, 3])

array(['<---1--->', '<---3--->'], dtype='<U9')


Por medio de un decorador, podemos definir `decorate_it` de la siguiente manera

In [109]:
@np.vectorize
def decorate_it(element):
    return decorate(element)

decorate_it([1, 3])

array(['<---1--->', '<---3--->'], dtype='<U9')

último problema nos queda: nos gustaría que el resultado final de `decorate_it` sea una lista, no un `numpy.array`. Para solucionar esto, definiremos la función `to_list`, que tome una función `f` y regrese una función de `f` evaluada y el resultado convertido a una lista.

In [118]:
def to_list(f):
    def fout(v):
        return list(f(v))
    return fout

@to_list
@np.vectorize
def decorate_it(string):
    return decorate(string)

decorate_it([1, 3])

['<---1--->', '<---3--->']

# _Decorators in the wild_

Lo siguiente que veremos son funciones comunmente usadas como decoradores dentro de la librería estándar de Python. Los primeros tres decoradores corresponden a funcionalidades adicionales que podemos otorgar a las clases.

## `@staticmethod`

Un `staticmethod`, como su nombre lo menciona, es un método el cuál no depende de la instancia de la clase desde la cuál fue invocada.

La sintáxis de un `staticmethod` es la siguiente:

```python
class A:
    @staticmethod
    def staticm(a, b):
        # so smth
        ...
```

Como podemos ver en el ejemplo, un `@staticmethod` no toma `self` como un primer parámetro. 

Comúnmente usamos `staticmethod`s cuando querramos definir funciones de una clase que no dependan de una instancia para ser invocadas.

<h2 style="color:teal">Ejemplo</h2>

Consideremos la clase `Normal` y supongamos nos interesa tener una función que valide si un `sigma` dado es un parámetro que podamos ocupar, i.e, $\sigma > 0$.

```python
import numpy as np
class Normal:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
    
    def pdf(self, x):
        return np.exp(-(x - self.mu) ** 2 / (2 * self.sigma ** 2)) / (2 * np.pi * self.sigma) ** 2
```

In [25]:
import numpy as np
class Normal:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
    
    def pdf(self, x):
        return np.exp(-(x - self.mu) ** 2 / (2 * self.sigma ** 2)) / np.sqrt(2 * np.pi * self.sigma ** 2)
    
    @staticmethod
    def is_valid_sigma(sigma):
        return True if sigma > 0 else False

In [22]:
norm = Normal(0, 1)
norm.pdf(0)

0.025330295910584444

Suponiendo que deseemos saber si un valor dado puede ser utilizado como parámetro `sigma`, no tendría por qué ser necesario crear una instancia de la clase `Normal` para validar esto. Sin embargo, es lógico querer definir una función que viva dentro de la clase `Normal` ya que la función está definida para el caso específico de un valor dentro de nuestra clase.

In [23]:
Normal.is_valid_sigma(2)

True

In [24]:
Normal.is_valid_sigma(-0.4)

False

## `@classmethod`

Un `@classmethod` es un método el cuál depende, como primer parámetro, de una clase.

```python
class A:
    @classmethod
    def method(cls, a, b):
        # do smth
        ...
```

La funcionalidad común de un `@classmethod` es definir una manera alternativa de inicializar una clase.

Considerando la clase `Normal` definida anteriormente, supongamos queremos inicializar nuestra clase de una manera diferente, en lugar de tener que pasar `sigma` como desviación estándard, nos gustaría inicializar la normal con un parámetro `beta` definida como la precisión $\beta = 1 / \sigma^2$.

$$
    p(x | \mu, \beta) = \frac{\beta}{\sqrt{2\pi}}\exp\left(-\frac{\beta}{2}(x - \mu) ^ 2\right)
$$

In [28]:
class Normal:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
    
    def pdf(self, x):
        return np.exp(-(x - self.mu) ** 2 / (2 * self.sigma ** 2)) / np.sqrt(2 * np.pi * self.sigma ** 2)
    
    @staticmethod
    def is_valid_sigma(sigma):
        return True if sigma > 0 else False
    
    @classmethod
    def from_precision(cls, mu, beta):
        sigma = 1 / np.sqrt(beta)
        return cls(mu, sigma)

In [30]:
norm1 = Normal(0, 10)
norm1.pdf(0)

0.03989422804014327

In [31]:
norm1 = Normal.from_precision(0, 10)
norm1.pdf(0)

1.2615662610100802

## `@property`

Como su nombre menciona, `@property` nos permite definir un _getter_ para elementos que deseemos tengan un nombre privado.


```python
class A:
    def __init__(self, param):
        self._param = param
    @property
    def param(self):
        return self._param
```

En la mayoría de las clases que definamos, como es el caso de la clase `Normal` que hemos estado definiendo, es necesario tener valores que no se puedan modificar una vez inicializada la función. Anteriormente vimos que una manera de resolver este problema es definiendo una función que nos regrese el valor de la variable. Otra manera en la que podemos lograr esto es por medio de `@property`.

In [91]:
class Normal:
    def __init__(self, mu, sigma):
        self._mu = mu
        self._sigma = sigma
    
    def pdf(self, x):
        return np.exp(-(x - self.mu) ** 2 / (2 * self.sigma ** 2)) / np.sqrt(2 * np.pi * self.sigma ** 2)
    
    @staticmethod
    def is_valid_sigma(sigma):
        return True if sigma > 0 else False
    
    @classmethod
    def from_precision(cls, mu, beta):
        sigma = 1 / np.sqrt(beta)
        return cls(mu, sigma)
    
    @property
    def mu(self):
        return self._mu
    
    @property
    def sigma(self):
        return self._sigma

In [92]:
norm0 = Normal(0, 1.23)
norm0.pdf(0)

0.3243433173995388

In [93]:
norm0.sigma

1.23

In [94]:
# Python nos prohibe acceder al valor de la clase y modificarla
norm0.sigma = 2

AttributeError: can't set attribute

## `ipywidgets`

In [90]:
from ipywidgets import interact, FloatSlider
import matplotlib.pyplot as plt

@interact(mu=FloatSlider(min=-1, max=1, value=0),
          variant=FloatSlider(min=0.1, max=2, value=1, step=0.1))
def plot(mu, variant):
    # Definiendo las distribuciones
    norm = Normal(mu, variant)
    norm_prec = Normal.from_precision(mu, variant)
    # Definiendo parámetros de las figuras
    fig, ax = plt.subplots(1, 2, figsize=(10, 3))
    x = np.linspace(-3, 3, 200)
    dists = [norm, norm_prec]
    params = [r"\sigma", r"\beta"]
    for axi, param, dist in zip(ax, params, dists):
        axi.set_title(f"Variant of ${param}$")
        axi.set_ylim(0, 0.8)
        axi.plot(x, dist.pdf(x))

interactive(children=(FloatSlider(value=0.0, description='mu', max=1.0, min=-1.0), FloatSlider(value=1.0, desc…

<h2 style="color:crimson">Ejercicios</h2>

1. Define una función `puntea` para un decorador que tome una función e imprima `---------` antes y después de que la función imprima su resultado

```python
>>> @puntea
>>> def saluda(nombre):
>>>    print(f"!Hola, {nombre}!")

>>> saluda("Gerardo")
---------
!Hola, Gerardo!
---------
```

---

2. La esperanza de una variable aleatoria discreta se define de la siguiente manera:

$$
    \mathbb{E}[x] = \sum_i x_i p(x_i)
$$

---

4. Considerando la clase `Human` añade el `classmethod` `from_birthday` que inicialice la clase considerando una fecha de nacimiento de tipo `datetime.datetime`.

```python
class Human:
    def __init__(self, name, last_name, age):
        self.name = name
        self.last_name = last_name
        self.age = age
```

---

5. Modifica la clase `Human`: implementa el `staticmethod` `is_adult` que valide si una persona es mayor de edad, i.e., edad >= 18.

---

6. Modifica la clase `Human`: implementa `property` a cada una de los parámetros dentro del constructor a fin que se pueda acceder, pero no modificar `name`, `last_name` y `age`.

## Referencias
1. https://www.python.org/dev/peps/pep-0318/